## Отческа в задаче классификации

### Рассмотрим один из часто забываемых гипермараметров в задачах классификации - работу с отсечкой (thresholding). 
Отсечка - это пороговое значение, которое определяет, какой класс будет выбран для каждого объекта. Несмотря на то, что отсечка может оказать значительное влияние на качество модели, она часто игнорируется в процессе обучения и оценки модели. Рассмотрим, как работать с отсечкой в задачах бинарной классификации, а также как выбирать оптимальное значение отсечки для достижения наилучшего качества модели

In [1]:
# импортируем нужные библиотеки
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_predict,
    cross_val_score)
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    classification_report,
    precision_score,
    recall_score,
    confusion_matrix)

import warnings
warnings.filterwarnings("ignore")

У нас есть тренировочные и тестовые данные о погода, а также значения целевого признака для тестовых данных (будет ли завтра дождь)

In [2]:
weather = pd.read_csv('train_dataset.csv')
X_test = pd.read_csv('test_dataset.csv')
y_test = pd.read_csv('answers.csv').drop('id', 1)

weather

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow
0,12.3,25.7,10.8,9.8,12.6,22.0,6.0,11.0,38.0,34.0,1018.4,1014.5,1.0,1.0,21.0,24.4,0
1,11.5,21.5,0.0,NaN,NaN,41.0,15.0,19.0,71.0,56.0,1018.5,1017.0,NaN,NaN,16.0,18.4,0
2,1.6,9.0,0.8,NaN,NaN,31.0,17.0,20.0,100.0,100.0,1024.4,1024.9,7.0,7.0,5.0,8.3,0
3,7.6,14.2,4.6,NaN,NaN,37.0,20.0,20.0,85.0,68.0,1017.9,1017.9,8.0,8.0,8.7,12.8,0
4,15.4,24.7,0.0,NaN,NaN,37.0,15.0,19.0,60.0,45.0,1010.3,1009.6,NaN,NaN,21.6,23.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52752,19.7,25.7,29.6,NaN,NaN,41.0,28.0,6.0,95.0,70.0,1016.4,1016.2,8.0,5.0,20.0,24.3,1
52753,11.2,26.5,0.0,4.0,8.9,33.0,13.0,20.0,45.0,23.0,1028.5,1024.4,6.0,4.0,21.3,25.9,0
52754,8.7,28.0,0.0,5.6,NaN,33.0,17.0,9.0,48.0,NaN,1017.9,1013.3,0.0,NaN,17.0,NaN,0
52755,1.0,16.7,0.2,NaN,NaN,44.0,0.0,17.0,100.0,54.0,1024.7,1020.0,8.0,NaN,4.3,16.4,0


Заменим пропущенные значения медианой, но только не в целевом признаке

In [3]:
medians_dict = {}

for col in weather.columns.drop('RainTomorrow'):
    medians_dict[col] = weather[col].median()

for col in weather.columns.drop('RainTomorrow'):
    weather[col] = weather[col].fillna(medians_dict[col])
    
for col in weather.columns.drop('RainTomorrow'):
    X_test[col] = X_test[col].fillna(medians_dict[col])

y_train = weather['RainTomorrow']
X_train = weather.drop('RainTomorrow', 1)

В итоге, получаем, что в среднем вероятность дождя составляет 21%

In [4]:
weather['RainTomorrow'].mean()

0.21045548458024527

Теперь проведем кросс-валидацию и найдем лучшие гиперпараметры модели случайного леса, будем ее тренировать под метрику ROC-AUC

In [5]:
parameters = {
    'n_estimators': [160, 320],
    'max_depth': [14],
    'max_features': [4]
}

In [6]:
rf = RandomForestClassifier()

In [7]:
clf = GridSearchCV(rf, parameters, scoring='roc_auc', cv=8, n_jobs=-1)

In [8]:
%%time
clf.fit(X_train, y_train)

Wall time: 1min 53s


GridSearchCV(cv=8, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [14], 'max_features': [4],
                         'n_estimators': [160, 320]},
             scoring='roc_auc')

In [9]:
clf.best_params_

{'max_depth': 14, 'max_features': 4, 'n_estimators': 320}

Получили, что лучшие гиперпараметры: максимальная глубина дерева - 14, кол-во деревьев - 320, кол-во признаков - 4

In [10]:
clf.best_estimator_

RandomForestClassifier(max_depth=14, max_features=4, n_estimators=320)

In [11]:
clf.best_score_

0.8713670426576272

Значение ROC-AUC довольно высокое - 0.87

Однако у нас присутствует дисбаланс классов (дождь не в 50% случаях, а в 21%), поэтому лучше смотреть на F1 меру и precision с recall отдельно

In [12]:
f1_score(y_test, clf.best_estimator_.predict(X_test.drop('id', 1)))

0.5741324921135647

In [13]:
precision_score(y_test, clf.best_estimator_.predict(X_test.drop('id', 1)))

0.7576568539994053

In [14]:
recall_score(y_test, clf.best_estimator_.predict(X_test.drop('id', 1)))

0.4621803011064756

In [15]:
print(classification_report(y_train, clf.best_estimator_.predict(X_train)))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     41654
           1       0.98      0.69      0.81     11103

    accuracy                           0.93     52757
   macro avg       0.95      0.84      0.88     52757
weighted avg       0.94      0.93      0.93     52757



In [16]:
print(confusion_matrix(y_train, clf.best_estimator_.predict(X_train)))

[[41505   149]
 [ 3440  7663]]


Получается, **recall** (0.46) гораздо ниже **precision** (0.76), однако именно он отвечает за способность модели правильно определить положительный класс. Тут нам поможет работа с отческой

### Выбор оптимальной отсечки классификатора

Возьмем вероятности 1 класса для каждого объекта с помощью метода ```.predict_proba```

In [17]:
predict = clf.best_estimator_.predict_proba(X_train)[:, 1]

In [18]:
predict

array([0.01839418, 0.1092682 , 0.1493935 , ..., 0.04588993, 0.09126177,
       0.06165302])

Стандартная отческа у нас 0.5

Мы рассмотрим все возможные отсечки от 0.01 до 0.99 и найдем, при каком значении **F1 мера** будет максимальной

In [19]:
predict > .5

array([False, False, False, ..., False, False, False])

In [20]:
%%time
predict = clf.best_estimator_.predict_proba(X_train)[:, 1]
best_thres = 0.01
max_f_score = 0
for i in range(1, 100):
    thres = i / 100
    var = f1_score(y_train, list(map(int, predict >= thres)))
    if var > max_f_score:
        max_f_score = var
        best_thres = thres

Wall time: 7.66 s


In [21]:
max_f_score

0.8616578615176025

In [22]:
best_thres

0.34

In [23]:
f1_score(
    y_test,
    list(
        map(int, clf.best_estimator_.predict_proba(X_test.drop("id", 1))[:, 1] >= 0.35)))

0.6309973294041809

In [24]:
precision_score(
    y_test,
    list(
        map(int, clf.best_estimator_.predict_proba(X_test.drop("id", 1))[:, 1] >= 0.35)))

0.6408529741863075

In [25]:
recall_score(
    y_test,
    list(
        map(int, clf.best_estimator_.predict_proba(X_test.drop("id", 1))[:, 1] >= 0.35)))

0.6214402321784872

Да, **precision** стал ниже, однако и **F1 мера**, и **recall** значительно повысили свои значения, а в данной задаче это важнее

Такой результат получился при отсечке в **0.34**